# Pipeline: Bronze -> Silver

## Instruções e informações

### Objetivo:  

Nesta fase, realizamos um processo de **extração, limpeza e transformação** dos dados:

- **Extração:** carregamento de arquivos CSV brutos provenientes de diferentes fontes.  
- **Limpeza:** tratamento de valores faltantes, padronização de tipos e formatos, correção de inconsistências.  
- **Transformação:** aplicação de regras de negócio, filtros, agregações e organização dos dados de forma estruturada para análise.  

### Configuração Inicial

Esta seção realiza a configuração inicial de todo o ambiente que será utilizado nas análises subsequentes, incluindo:  

- Importação das bibliotecas necessárias (ex.: `pandas`, `numpy`, `matplotlib`, `seaborn`), explicando a função de cada uma na sequência de transformações.  
- Definição dos caminhos para os arquivos de dados brutos (CSV da camada Bronze) que serão processados.  
- Identificação de metadados relevantes que podem ser úteis para a limpeza e análise, como tipos de colunas, valores nulos, formatos de data, etc.  


In [ ]:
import pandas as pd
import numpy as np
import os

# --- 1. Definições de Caminho e Arquivos ---

BASE_PATH = '../data/bronze_raw/' 

file_names = [
    'circuits.csv',
    'constructor_results.csv',
    'constructor_standings.csv',
    'constructors.csv',
    'driver_standings.csv',
    'drivers.csv',
    'lap_times.csv',
    'pit_stops.csv',
    'qualifying.csv',
    'races.csv',
    'results.csv',
    'seasons.csv',
    'sprint_results.csv',
    'status.csv'
]

df_bronze = {}

# --- 2. Função de Carregamento e Tratamento Inicial ---

def load_bronze_data(file_name):
    """
    Carrega um arquivo CSV da Camada Bronze, substituindo '\\N' por NaN 
    e forçando a leitura de IDs como string para evitar problemas de tipo.
    """
    file_path = os.path.join(BASE_PATH, file_name)
    print(f"Carregando: {file_name}...")
    
    # Define as chaves comuns que devem ser lidas como strings (object)
    dtype_map = {
        col: 'object' for col in ['driverId', 'raceId', 'constructorId', 'statusId', 'circuitId'] 
        if col in pd.read_csv(file_path, nrows=0).columns
    }
    
    df = pd.read_csv(
        file_path, 
        na_values=['\\N'], 
        dtype=dtype_map,
        encoding='utf-8'
    )
    
    print(f"  -> Total de {len(df)} registros carregados.")
    return df


# --- 3. Execução do Carregamento ---

for file in file_names:
    # A chave do dicionário será o nome do arquivo sem a extensão '.csv'
    df_name = file.replace('.csv', '')
    try:
        df_bronze[df_name] = load_bronze_data(file)
    except FileNotFoundError:
        print(f"ERRO: Arquivo {file} não encontrado no caminho {BASE_PATH}. Pulando.")
    except Exception as e:
        print(f"ERRO ao carregar {file}: {e}")
        
print("\n--- Carregamento Concluído ---")
print(f"DataFrames disponíveis no dicionário 'df_bronze': {list(df_bronze.keys())}")

Carregando: circuits.csv...
  -> Total de 77 registros carregados.
Carregando: constructor_results.csv...
  -> Total de 12625 registros carregados.
Carregando: constructor_standings.csv...
  -> Total de 13391 registros carregados.
Carregando: constructors.csv...
  -> Total de 212 registros carregados.
Carregando: driver_standings.csv...
  -> Total de 34863 registros carregados.
Carregando: drivers.csv...
  -> Total de 861 registros carregados.
Carregando: lap_times.csv...
  -> Total de 589081 registros carregados.
Carregando: pit_stops.csv...
  -> Total de 11371 registros carregados.
Carregando: qualifying.csv...
  -> Total de 10494 registros carregados.
Carregando: races.csv...
  -> Total de 1125 registros carregados.
Carregando: results.csv...
  -> Total de 26759 registros carregados.
Carregando: seasons.csv...
  -> Total de 75 registros carregados.
Carregando: sprint_results.csv...
  -> Total de 360 registros carregados.
Carregando: status.csv...
  -> Total de 139 registros carregad

___
## Processo de Transformação para a Camada SILVER

Este notebook documenta a etapa de transformação de dados para o **Modelo Dimensional da Camada Silver**, sucedendo a ingestão de 14 arquivos CSV da base de dados de Fórmula 1 (Ergast) na Camada Bronze. Os dados brutos, preservados com granularidade transacional por corrida, piloto ou volta, são lidos integralmente e reestruturados para análise otimizada. As transformações essenciais incluem a desnormalização de atributos descritivos em **Dimensões coerentes** (`DIM_PILOTO`, `DIM_CORRIDA`, `DIM_CONSTRUTOR`), a criação e atribuição de **Chaves Substitutas (`_sk_`)** para todas as dimensões, e a consolidação de diferentes fontes de resultados (`results.csv`, `sprint_results.csv`, `constructor_results.csv`, `driver_standings.csv`, `constructor_standings.csv`) em Tabelas Fato específicas (`FATO_RESULTADOS_CORRIDA`, `FATO_POSICOES_CAMPEONATO`), promovendo uma visão unificada da performance.

Do ponto de vista de modelagem, adota-se o esquema Estrela/Snowflake com granularidade por resultado de corrida/piloto e por posição de campeonato. A transformação de chaves naturais (`driverId`, `raceId`) em chaves substitutas (`piloto_sk`, `corrida_sk`) otimiza as operações de *join* e permite a manutenção de histórico de atributos (SCD - Slowly Changing Dimensions). A ligação entre fatos e dimensões é padronizada via `_sk_` (e.g., `FATO_RESULTADOS_CORRIDA` liga-se à `DIM_PILOTO` via `#piloto_sk`), exceto pelo `#qualifyId`, que é mantido como **Dimensão Degenerada (DD)** no Fato, fornecendo o identificador único da qualificação diretamente no registro de resultado da corrida, o que viabiliza um Drill-Through preciso para os tempos de Q1/Q2/Q3 da Camada Bronze.

A governança do processo é assegurada pela preservação das **chaves naturais originais (`#Id`)** em todas as dimensões da Silver, garantindo a rastreabilidade completa e a capacidade de auditoria (*Drill-Through*) de volta para a Camada Bronze, em conformidade com a arquitetura Medallion. A tipagem explícita dos dados e a criação de campos derivados (como `nome_completo` na `DIM_PILOTO` ou o *flag* `is_sprint` no Fato) evitam ambiguidades e enriquecem o contexto analítico. Ao final, o resultado é materializado nas tabelas Fato e Dimensão com um modelo otimizado, prontas para serem consumidas por rotinas de agregação (na Camada Gold) e ferramentas de BI, estabelecendo uma base sólida para monitoramento perene da performance e resultados da Fórmula 1.

## Iniciando as Tabelas Fato

Esta seção da Camada Silver foca na criação das duas principais tabelas fato analíticas. Primeiramente, a `FATO_RESULTADOS_CORRIDA` é construída pela união dos resultados de Grandes Prêmios e Corridas Sprint, e é enriquecida com o `qualifyId` (Dimensão Degenerada). Em seguida, a `FATO_POSICOES_CAMPEONATO` é criada consolidando os rankings de pilotos e construtores. Ambas as tabelas recebem um padrão de nomenclatura em português, têm suas colunas tipadas corretamente e utilizam as chaves naturais da Camada Bronze como referência temporária.

In [16]:
# Certifique-se de que o dicionário df_bronze foi carregado corretamente.

if 'df_bronze' not in locals() or not df_bronze:
    print("ERRO: O dicionário 'df_bronze' não foi encontrado ou está vazio. Por favor, execute a célula de carregamento da Camada Bronze primeiro.")
else:
    
    # --- 1. CRIAÇÃO DA FATO_RESULTADOS_CORRIDA ---
    
    print("1. Criando FATO_RESULTADOS_CORRIDA...")
    
    # 1.1 Preparar resultados de GP (results)
    df_results_gp = df_bronze['results'].copy()
    df_results_gp['is_sprint'] = 0  # Flag: 0 = Grande Prêmio
    
    # 1.2 Preparar resultados de Sprint (sprint_results)
    df_results_sprint = df_bronze['sprint_results'].copy()
    df_results_sprint['is_sprint'] = 1  # Flag: 1 = Corrida Sprint
    
    # CORREÇÃO CRÍTICA: Adicionar colunas ausentes no Sprint para que a concatenação funcione.
    missing_cols_in_sprint = ['rank', 'fastestLapSpeed', 'time', 'fastestLapTime']
    for col in missing_cols_in_sprint:
        if col not in df_results_sprint.columns:
            df_results_sprint[col] = np.nan
            
    # 1.3 Definir colunas comuns para concatenação e renomear para o padrão Silver (BRZ_ID)
    cols_to_keep = [
        'resultId', 'raceId', 'driverId', 'constructorId', 'statusId', 
        'grid', 'positionOrder', 'points', 'laps', 
        'milliseconds', 'fastestLap', 'rank', 'fastestLapSpeed', 
        'position', 'time', 'fastestLapTime', 
        'is_sprint'
    ]
    
    df_results_gp = df_results_gp[cols_to_keep].rename(columns={'resultId': 'resultado_brz_id'})
    df_results_sprint = df_results_sprint[cols_to_keep].rename(columns={'resultId': 'resultado_brz_id'})
    
    df_fato_resultados = pd.concat([df_results_gp, df_results_sprint], ignore_index=True)
    
    # 1.4 Adicionar a Dimensão Degenerada (DD): qualifyId
    df_qualifying = df_bronze['qualifying'][['raceId', 'driverId', 'qualifyId']].copy()
    
    # Merge para trazer o qualifyId (DD).
    df_fato_resultados = pd.merge(
        df_fato_resultados,
        df_qualifying,
        on=['raceId', 'driverId'],
        how='left'
    )
    
    # 1.5 Renomeação e Mapeamento para o Fato Silver (Seguindo o MER)
    df_fato_resultados = df_fato_resultados.rename(columns={
        # Chaves estrangeiras (temporariamente BRZ IDs) e DD
        'raceId': '#corrida_sk',
        'driverId': '#piloto_sk',
        'constructorId': '#construtor_sk',
        'statusId': '#status_sk',
        'qualifyId': '#qualificacao_sk', # Usando o nome da SK para fins de mapeamento final
        
        # Atributos e Métricas
        'grid': '#grid',
        'rank': '#rank',
        'points': 'pontos',
        'positionOrder': 'posicao_final_ordenada',
        'laps': 'voltas_completadas',
        'milliseconds': 'tempo_total_ms',
        'fastestLap': 'volta_mais_rapida', # Número da volta mais rápida
        'fastestLapSpeed': 'velocidade_maxima',
        
        # Outras colunas
        'resultado_brz_id': 'resultado_brz_id_auditoria'
    }).drop(columns=['position', 'time', 'fastestLapTime'])
    
    # 1.6 Tipagem de Colunas
    numeric_cols = [
        'posicao_final_ordenada', 'pontos', 'tempo_total_ms', 'voltas_completadas', 
        'volta_mais_rapida', '#rank', 'velocidade_maxima', '#grid', 'is_sprint'
    ]
    for col in numeric_cols:
        if col in df_fato_resultados.columns:
            df_fato_resultados[col] = pd.to_numeric(df_fato_resultados[col], errors='coerce')
    
    # 1.7 Adicionar Chave Substituta (SK)
    df_fato_resultados['resultado_sk'] = range(1, len(df_fato_resultados) + 1)
    
    # 1.8 Selecionar e reordenar colunas finais para o Fato
    FATO_RESULTADOS_COLS = [
        'resultado_sk', '#corrida_sk', '#piloto_sk', '#construtor_sk', '#status_sk', 
        '#qualificacao_sk', '#grid', '#rank', 'pontos', 'posicao_final_ordenada', 
        'voltas_completadas', 'tempo_total_ms', 'volta_mais_rapida', 'velocidade_maxima', 
        'is_sprint', 'resultado_brz_id_auditoria'
    ]
    df_fato_resultados = df_fato_resultados[[col for col in FATO_RESULTADOS_COLS if col in df_fato_resultados.columns]].copy()
    
    
    # --- 2. CRIAÇÃO DA FATO_PONTUACAO_CAMPEONATO ---
    
    print("2. Criando FATO_PONTUACAO_CAMPEONATO...")

    # 2.1 Preparar Standings de Pilotos
    df_driver_standings = df_bronze['driver_standings'].copy()
    df_driver_standings['constructorId'] = np.nan # Preenche coluna de Construtor
    
    # 2.2 Preparar Standings de Construtores
    df_constructor_standings = df_bronze['constructor_standings'].copy()
    df_constructor_standings['driverId'] = np.nan # Preenche coluna de Piloto
    
    # 2.3 Definir colunas e concatenar
    standing_cols_base = ['raceId', 'driverId', 'constructorId', 'points', 'position', 'wins']

    df_driver_standings_sub = df_driver_standings.reindex(columns=standing_cols_base + ['driverStandingsId']).rename(
        columns={'driverStandingsId': 'standing_brz_id'}
    )
    df_constructor_standings_sub = df_constructor_standings.reindex(columns=standing_cols_base + ['constructorStandingsId']).rename(
        columns={'constructorStandingsId': 'standing_brz_id'}
    )
    
    df_fato_posicoes = pd.concat([
        df_driver_standings_sub, 
        df_constructor_standings_sub
    ], ignore_index=True)

    # 2.4 Renomeação e Mapeamento para o Fato Silver (Seguindo o MER)
    df_fato_posicoes = df_fato_posicoes.rename(columns={
        'raceId': '#corrida_sk',
        'driverId': '#piloto_sk',
        'constructorId': '#construtor_sk',
        'points': 'total_pontos',
        'position': 'posicao_campeonato',
        'wins': 'total_vitorias',
        'standing_brz_id': 'standing_brz_id_auditoria'
    })
    
    # Conversão de tipos numéricos
    numeric_standing_cols = ['total_pontos', 'posicao_campeonato', 'total_vitorias']
    for col in numeric_standing_cols:
        df_fato_posicoes[col] = pd.to_numeric(df_fato_posicoes[col], errors='coerce')

    # 2.5 Adicionar chave substituta (SK)
    df_fato_posicoes['posicao_sk'] = range(1, len(df_fato_posicoes) + 1)
    
    # 2.6 Selecionar e reordenar colunas finais para o Fato
    FATO_PONTUACAO_COLS = [
        'posicao_sk', '#corrida_sk', '#piloto_sk', '#construtor_sk', 
        'total_pontos', 'posicao_campeonato', 'total_vitorias', 
        'standing_brz_id_auditoria'
    ]
    df_fato_posicoes = df_fato_posicoes[[col for col in FATO_PONTUACAO_COLS if col in df_fato_posicoes.columns]].copy()
    
    
    # 2.7 Armazenamento na Camada Silver (Dicionário)
    # Define df_silver globalmente para a próxima célula funcionar
    if 'df_silver' not in locals():
        df_silver = {} 
    df_silver['fato_resultados_corrida'] = df_fato_resultados
    df_silver['fato_pontuacao_campeonato'] = df_fato_posicoes

    print("\n--- Criação das Tabelas Fato da Camada Silver Concluída ---")
    print(f"FATO_RESULTADOS_CORRIDA: {len(df_silver['fato_resultados_corrida'])} registros")
    print(f"FATO_PONTUACAO_CAMPEONATO: {len(df_silver['fato_pontuacao_campeonato'])} registros")
    print("Os DataFrames estão disponíveis no dicionário 'df_silver'.")

1. Criando FATO_RESULTADOS_CORRIDA...
2. Criando FATO_PONTUACAO_CAMPEONATO...

--- Criação das Tabelas Fato da Camada Silver Concluída ---
FATO_RESULTADOS_CORRIDA: 27119 registros
FATO_PONTUACAO_CAMPEONATO: 48254 registros
Os DataFrames estão disponíveis no dicionário 'df_silver'.


In [10]:
print("\n--- FATO_RESULTADOS_CORRIDA (.head()) ---")
print(df_silver['fato_resultados_corrida'].head())


--- FATO_RESULTADOS_CORRIDA (.head()) ---
   resultado_sk #corrida_sk #piloto_sk #construtor_sk #status_sk  \
0             1          18          1              1          1   
1             2          18          2              2          1   
2             3          18          3              3          1   
3             4          18          4              4          1   
4             5          18          5              1          1   

   #qualificacao_sk  #grid  #rank  pontos  posicao_final_ordenada  \
0               1.0      1    2.0    10.0                       1   
1               5.0      5    3.0     8.0                       2   
2               7.0      7    5.0     6.0                       3   
3              12.0     11    7.0     5.0                       4   
4               3.0      3    1.0     4.0                       5   

   voltas_completadas  tempo_total_ms  volta_mais_rapida  velocidade_maxima  \
0                  58       5690616.0               39

In [14]:
print("\n--- FATO_POSICOES_CAMPEONATO (.head()) ---")
print(df_silver['fato_posicoes_campeonato'].head())


--- FATO_POSICOES_CAMPEONATO (.head()) ---
   standing_brz_id race_id_brz driver_id_brz constructor_id_brz  total_pontos  \
0                1          18             1                NaN          10.0   
1                2          18             2                NaN           8.0   
2                3          18             3                NaN           6.0   
3                4          18             4                NaN           5.0   
4                5          18             5                NaN           4.0   

   posicao_campeonato_rank  total_vitorias tipo_standing  \
0                        1               1        PILOTO   
1                        2               0        PILOTO   
2                        3               0        PILOTO   
3                        4               0        PILOTO   
4                        5               0        PILOTO   

  posicao_campeonato_texto  posicao_sk  
0                        1           1  
1                        2

___
## Iniciando as Tabelas Dimensões

Esta etapa da Camada Silver foca na desnormalização das tabelas mestras da Camada Bronze em entidades dimensionais otimizadas. O processo envolve a criação de Chaves Substitutas (`_sk_`) como chaves primárias para todas as dimensões (`DIM_STATUS`, `DIM_CIRCUITO`, `DIM_PILOTO`, etc.), a renomeação de atributos para o português e a garantia da unicidade e integridade dos registros. As chaves naturais originais são preservadas como atributos de auditoria, preparando as dimensões para o join eficiente com as tabelas fato e para a construção final do Modelo Dimensional.

In [19]:
# Verificar se o dicionário df_bronze foi carregado corretamente.
if 'df_bronze' not in locals() or not df_bronze:
    print("ERRO: O dicionário 'df_bronze' não foi encontrado ou está vazio. Por favor, execute a célula de carregamento da Camada Bronze primeiro.")
else:
    if 'df_silver' not in locals():
        df_silver = {}
        
    print("--- INICIANDO CRIAÇÃO DAS TABELAS DIMENSÃO ---")

    # --- 1. DIM_STATUS ---
    print("1. Criando DIM_STATUS...")
    df_status = df_bronze['status'].copy()
    df_status = df_status.rename(columns={
        'statusId': '#statusId',
        'status': 'descricao_status'
    })
    df_status['_status_sk_'] = range(1, len(df_status) + 1)
    df_status = df_status[['_status_sk_', '#statusId', 'descricao_status']].copy()
    df_silver['dim_status'] = df_status

    # --- 2. DIM_CIRCUITO ---
    print("2. Criando DIM_CIRCUITO...")
    df_circuitos = df_bronze['circuits'].copy()
    df_circuitos = df_circuitos.rename(columns={
        'circuitId': '#circuitId',
        'name': 'nome',
        'location': 'localidade',
        'country': 'país',
        'lat': 'latitude',
        'lng': 'longitude',
        'alt': 'altitude'
    }).drop(columns=['circuitRef', 'url'])

    numeric_cols_circuito = ['latitude', 'longitude', 'altitude']
    for col in numeric_cols_circuito:
        df_circuitos[col] = pd.to_numeric(df_circuitos[col], errors='coerce')

    df_circuitos['_circuito_sk_'] = range(1, len(df_circuitos) + 1)
    df_circuitos = df_circuitos[['_circuito_sk_', '#circuitId', 'nome', 'localidade', 'país', 'latitude', 'longitude', 'altitude']].copy()
    df_silver['dim_circuito'] = df_circuitos

    # --- 3. DIM_CONSTRUTOR ---
    print("3. Criando DIM_CONSTRUTOR...")
    df_construtor = df_bronze['constructors'].copy()
    df_construtor = df_construtor.rename(columns={
        'constructorId': '#constructorId',
        'name': 'nome',
        'nationality': 'nacionalidade'
    }).drop(columns=['constructorRef', 'url'])
    
    df_construtor['_constructor_sk_'] = range(1, len(df_construtor) + 1)
    df_construtor = df_construtor[['_constructor_sk_', '#constructorId', 'nome', 'nacionalidade']].copy()
    df_silver['dim_construtor'] = df_construtor
    
    # --- 4. DIM_PILOTO ---
    print("4. Criando DIM_PILOTO...")
    df_piloto = df_bronze['drivers'].copy()
    
    df_piloto = df_piloto.rename(columns={
        'driverId': '#driverId',
        'forename': 'nome',
        'surname': 'sobrenome',
        'dob': 'data_nascimento',
        'nationality': 'nacionalidade',
        'code': 'codigo'
    }).drop(columns=['driverRef', 'number', 'url'])

    df_piloto['nome_completo'] = df_piloto['nome'] + ' ' + df_piloto['sobrenome']  
    df_piloto['data_nascimento'] = pd.to_datetime(df_piloto['data_nascimento'], errors='coerce')
    df_piloto['_piloto_sk_'] = range(1, len(df_piloto) + 1)
    df_piloto = df_piloto[['_piloto_sk_', '#driverId', 'nome', 'sobrenome', 'data_nascimento', 'nacionalidade', 'codigo']].copy()
    df_silver['dim_piloto'] = df_piloto

    # --- 5. DIM_QUALIFICACAO ---
    print("5. Criando DIM_QUALIFICACAO...")
    df_qualificacao = df_bronze['qualifying'].copy()
    
    df_qualificacao = df_qualificacao.rename(columns={
        'qualifyId': '#qualifyId',
        'raceId': '#raceId',
        'driverId': '#driverId',
        'position': 'posicao_grid',
        'q1': 'tempo_q1',
        'q2': 'tempo_q2',
        'q3': 'tempo_q3'
    }).drop(columns=['constructorId', 'number'])

    df_qualificacao['posicao_grid'] = pd.to_numeric(df_qualificacao['posicao_grid'], errors='coerce')

    df_qualificacao['_qualificacao_sk_'] = range(1, len(df_qualificacao) + 1)
    df_qualificacao = df_qualificacao[['_qualificacao_sk_', '#qualifyId', '#raceId', '#driverId', 'posicao_grid', 'tempo_q1', 'tempo_q2', 'tempo_q3']].copy()
    df_silver['dim_qualificacao'] = df_qualificacao
    
    # --- 6. DIM_CORRIDA (Com Snowflake para CIRCUITO) ---
    print("6. Criando DIM_CORRIDA...")
    df_corrida = df_bronze['races'].copy()
    
    # 6.1 Tratamento de data/hora
    df_corrida['date'] = pd.to_datetime(df_corrida['date'], errors='coerce')
    
    # 6.2 Determinação do tipo de evento (Simplificada)
    df_corrida['tipo_evento'] = 'GP'
    df_corrida.loc[df_corrida['sprint_date'].notna() | df_corrida['sprint_time'].notna(), 'tipo_evento'] = 'SPRINT_WEEKEND'
    
    df_corrida = pd.merge(
        df_corrida,
        df_silver['dim_circuito'][['#circuitId', '_circuito_sk_']],
        left_on='circuitId',
        right_on='#circuitId',
        how='left'
    )
    
    df_corrida = df_corrida.drop(columns=['#circuitId'])

    # 6.3 Renomeação e Limpeza
    df_corrida = df_corrida.rename(columns={
        'raceId': '#raceId',
        'circuitId': 'circuitId_brz', # Mantém a chave natural do circuito para auditoria
        'year': 'ano',
        'round': 'rodada',
        'name': 'nome_gp',
        'date': 'data_gp',
        '_circuito_sk_': '#circuito_sk' # Chave estrangeira Snowflake
    }).drop(columns=[c for c in df_corrida.columns if c.startswith(('fp1', 'fp2', 'fp3', 'quali', 'sprint', 'time', 'url'))])

    df_corrida['_corrida_sk_'] = range(1, len(df_corrida) + 1)
    df_corrida = df_corrida[['_corrida_sk_', '#raceId', '#circuito_sk', 'ano', 'rodada', 'nome_gp', 'data_gp', 'tipo_evento']].copy()
    df_silver['dim_corrida'] = df_corrida

--- INICIANDO CRIAÇÃO DAS TABELAS DIMENSÃO ---
1. Criando DIM_STATUS...
2. Criando DIM_CIRCUITO...
3. Criando DIM_CONSTRUTOR...
4. Criando DIM_PILOTO...
5. Criando DIM_QUALIFICACAO...
6. Criando DIM_CORRIDA...


In [ ]:
print("\n--- DIM_PILOTO (.head()) ---")
print(df_silver['dim_piloto'].head())


--- DIM_PILOTO (.head()) ---
   _piloto_sk_ #driverId      nome   sobrenome data_nascimento nacionalidade  \
0            1         1     Lewis    Hamilton      1985-01-07       British   
1            2         2      Nick    Heidfeld      1977-05-10        German   
2            3         3      Nico     Rosberg      1985-06-27        German   
3            4         4  Fernando      Alonso      1981-07-29       Spanish   
4            5         5    Heikki  Kovalainen      1981-10-19       Finnish   

  codigo  
0    HAM  
1    HEI  
2    ROS  
3    ALO  
4    KOV  


In [ ]:
print("\n--- DIM_CORRIDA (.head()) ---")
print(df_silver['dim_corrida'].head())


--- DIM_CORRIDA (.head()) ---
   _corrida_sk_ #raceId  #circuito_sk   ano  rodada                nome_gp  \
0             1       1             1  2009       1  Australian Grand Prix   
1             2       2             2  2009       2   Malaysian Grand Prix   
2             3       3            17  2009       3     Chinese Grand Prix   
3             4       4             3  2009       4     Bahrain Grand Prix   
4             5       5             4  2009       5     Spanish Grand Prix   

     data_gp tipo_evento  
0 2009-03-29          GP  
1 2009-04-05          GP  
2 2009-04-19          GP  
3 2009-04-26          GP  
4 2009-05-10          GP  


___
## Salvando os dados 

Nesta etapa, as tabelas Fato e Dimensão que foram criadas e estruturadas em DataFrames do Pandas na Camada Silver serão persistidas. Cada DataFrame será exportado para um arquivo CSV único, utilizando o ponto e vírgula (;) como separador e garantindo a codificação UTF-8 para preservar acentuações e caracteres especiais. O atributo de índice de linha do Pandas será descartado para manter os arquivos limpos e alinhados com a estrutura de tabelas relacionais.

In [ ]:
BASE_SILVER_PATH = '../data/silver/'

if 'df_silver' not in locals() or not df_silver:
    print("ERRO: O dicionário 'df_silver' não foi encontrado ou está vazio. Por favor, execute as células de criação das tabelas Fato e Dimensão.")
else:
    print("\n--- Iniciando Materialização das Tabelas Silver para CSV ---")

    for table_name, df in df_silver.items():
        file_path = os.path.join(BASE_SILVER_PATH, f"{table_name}.csv")
        
        # Salva o DataFrame como um arquivo CSV
        df.to_csv(
            file_path,
            index=False,        # Não salva o índice do DataFrame
            sep=';',            # Usa ponto e vírgula como separador
            encoding='utf-8'    # Garante a codificação correta
        )
        print(f"Materializado: '{table_name}.csv' -> {len(df)} registros.")

    print("\n--- Materialização da Camada Silver Concluída ---")


--- Iniciando Materialização das Tabelas Silver para CSV ---
Materializado: 'fato_resultados_corrida.csv' -> 27119 registros.
Materializado: 'fato_posicoes_campeonato.csv' -> 48254 registros.
Materializado: 'fato_pontuacao_campeonato.csv' -> 48254 registros.
Materializado: 'dim_status.csv' -> 139 registros.
Materializado: 'dim_circuito.csv' -> 77 registros.
Materializado: 'dim_construtor.csv' -> 212 registros.
Materializado: 'dim_piloto.csv' -> 861 registros.
Materializado: 'dim_qualificacao.csv' -> 10494 registros.
Materializado: 'dim_corrida.csv' -> 1125 registros.

--- Materialização da Camada Silver Concluída ---
